## Title: EDA of Welltory COVID-19 and Wearables Open Data Research
### Team name: Deftones
### Team Members: Tal Erez, Ahmed Boutar 



#### Submission Description:
The notebook deftones-eda-ta4.ipynb is used to perform EDA on the Welltory COVID-19 and Wearables dataset. Since we did not choose a specific problem or question to answer, we decided to work on most of the datasets provided to get more practice doing eda and working with multiple tables. This notebook includes:
- A discussion on data quality assessment, including data profiling, data completeness, data accuracy, data consistency, data integrity, and data lineage and provenance
- Exploration and interpretation of the data structure, descriptive statistics, data quality, and variable relationships
- Visualizations of the different datasets
- Implementation of strategies for Handling Missing Values, Removing Duplicates, and Handling Outliers for some of the datasets 
- Data transformations for some of the tables 
- Creating a new feature for the participants table
- Implementation of a dimensionality reduction method on the blood pressure table

## Instructions to run:
### 1. Download the datasets from https://github.com/Welltory/hrv-covid19/tree/master/data
### 2. Create folder names 'data' and move the csv files in there. The directory you are on should look like:
```
project-root/
├── data/
│   └── blood_pressure.csv
│   └── heart_rate.csv
│   └── hrv_measurements.csv
│   └── participants.csv
│   └── scales_description.csv
│   └── sleep.csv
│   └── surveys.csv
│   └── wearables.csv
│   └── weather.csv
└── deftones-eda-ta4.ipynb
```
### 3. Change the directory to where the deftones-eda-ta4.ipynb is
### 4. Create a requirements.txt
### 5. Paste the dependencies included above in the requirements.txt file
### 6. Set-up and activate the virtual environment:
Mac OS/Linux:
```
python3 -m venv venv
source venv/bin/activate
```
Windows:
```
python -m venv venv
venv\Scripts\activate
```
### 7. Install the required dependencies
```
pip install -r requirements.txt
```
### 8. Open the notebook file and select the kernel you would like to use in the top right (the venv you just created)
### 9. Run each cell (can be done using shift + enter/return)

### Introduction
In this notebook, we will analyze the dataset titles "Welltory COVID-19 and Wearables Open Data Research". This dataset is part of a research carried out in 2020. 

The goal behind this dataset is to "detect patterns regarding the COVID-19 disease; progression and recovery"

The dataset, created by the Welltory team, was made available to the public on a non-commercial basis in an effort to fight the pandemic

Source: https://github.com/Welltory/hrv-covid19/tree/master?tab=readme-ov-file

In [125]:
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

In [126]:
#Configuring the output of pd so it doesn't truncate columns
pd.set_option('display.max_columns', None)
pd.set_option("expand_frame_repr", False)

# Initial Data Exploration & Assessment

### Data Lineage & Provenance

The dataset provided contains data of users with positive COVID-19 that agreed to participate in the research. The research involved tracking the users' (participants) symptomps, heart rate variability, and other data, which involved using Welltory wearables that keep track of the different metrics, accessible through the Welltory app.

The data collected by the researchers involved: 
- "Heart rate variability measurements. Measurements were made with any Bluetooth-enabled heart rate monitor or with a smartphone camera with a high resolution - a method called Photoplethysmography (PPG). It is a simple optical technique used to detect blood volume changes in the microvascular bed of tissue to track the heartbeat. 

- Data from user-connected gadgets including devices such as Apple Watch and Garmin that sync with Google Fit or Apple Health.

- Clinically validated physical and mental health assessments. We created a feature specifically for this project, where people would add information about symptoms and test results." (Source: https://github.com/Welltory/hrv-covid19/tree/master?tab=readme-ov-file)

Since the main data the research is based on is the HRV measurement, it is important to note that getting readings to estimate HRV have many factors that could influence PPG results such as the person's movements or hardware issues, which may raise questions about the data quality. 

The Welltory Team published a paper titled "Wavelet Analysis And Self-Similarity Of Photoplethysmography Signals For HRV Estimation And Quality Assessment", where they discuss their strategy to collect the data ensuring accurate readings, as well as documenting their signal processing process. To be specific, they introduced a new algorithm that does "not only detect peaks, but also identify corrupted signal parts. Their prepocessing procesure included specific steps to avoid signal issues such as "vanished signal and abrupt shift". Although the researchers seem to have taken appropriate steps to report the most accurate measurements, they mention that their algorithm demonstrates one limitation. In fact, they conclude that "the algorithm perform well on various PPG signals", but "cannot be used for HRV estimation from PPG signals collected during or right after exercise since [in which case] the PPG signal does not contain sufficient information". (Source: https://www.mdpi.com/1424-8220/21/20/6798#sec3dot1-sensors-21-06798)

The dataset is split among 9 different csv files namely: 
- participants.csv: containing general information about users (participants)
- hrv_measurements.csv: contains data based on heart rate variability (HRV) measurements collected from COVID-19 participants via the Welltory app
- blood_pressure.csv: Contains blood pressure data and derivatives. Fields functional_changes_index, circulatory_efficiency, kerdo_vegetation_index, robinson_index are calculated when heart rate data is available during measurement
- heart_rate.csv: contains raw heart rate intervals
- wearables.csv: Contains data collected from supported gadgets and aggregated by day
- sleep.csv: Contains data about user sleep collected from supported gadgets and aggregated by day
- weather.csv: Contains data about weather conditions for user's location aggregated by day
- surveys.csv: Contains results of health-related surveys that users take in Welltory app
- scales_description: Contains scales description, value, and meaning

To learn more about the columns in each file, please refer to the datatypes.md file provided by the researchers

# Data Profiling (EDA)

In [ ]:
df_blood_pressure = pd.read_csv("data/blood_pressure.csv")
print(df_blood_pressure.head())

In [ ]:
df_heart_rate = pd.read_csv("data/heart_rate.csv")
df_heart_rate.head()

In [ ]:
df_hrv_measurements = pd.read_csv("data/hrv_measurements.csv")
df_hrv_measurements.head()

In [ ]:
df_participants = pd.read_csv("data/participants.csv")
df_participants.head()

In [ ]:
df_scales_description = pd.read_csv("data/scales_description.csv")
df_scales_description.head()

In [ ]:
df_sleep = pd.read_csv("data/sleep.csv")
df_sleep.head()

In [ ]:
df_surveys = pd.read_csv("data/surveys.csv")
df_surveys.head()

In [ ]:
df_wearables = pd.read_csv("data/wearables.csv")
df_wearables.head()

In [ ]:
df_weather = pd.read_csv("data/weather.csv")
df_weather.head()

Some of the steps taken in the EDA were inspired by Dr.Brinnae Bent's EDA code demo from an in-lecture video

### Data Structure

In [136]:
df_list = [df_blood_pressure, df_heart_rate, df_hrv_measurements, df_participants, df_scales_description, df_sleep, df_surveys, df_wearables, df_weather]
df_list_names = ["df_blood_pressure", "df_heart_rate", "df_hrv_measurements", "df_participants", "df_scales_description", "df_sleep", "df_surveys", "df_wearables", "df_weather"]

In [ ]:
for i, df_name in enumerate(df_list_names):
    #Data Structure
    print(f"Data Structure for {df_name}")
    print("-"*10)
    print(f"Dimensions: {df_list[i].shape}")
    print(f"Data Types:\n{df_list[i].dtypes}")
    print(f"Missing Values:\n{df_list[i].isnull().sum()}")
    print(f"Unique observations:\n{df_list[i].nunique()}")
    print('\n')

### Interpretation

##### Blood Pressure
The blood pressure table contains 721 observations of users' (participants) blood pressure, with each observation having multiple features that relate to blood pressure measurements that take into account different factors (e.g functional_changes_indes is an assessment of how well the body can adapt to stresors. Take height, weight, and gender into account). **It is interesting to note that the height, weight, and gender are features of the participants table**.

The numerical features represent the blood pressure measurements (diastolic and systolic) as well as other measurements that assess how well the body react to stressors and the blood circulation efficiency among other things.  

The categorical features represent the user id and the day-time for when this observation was made. **It is important to note that these measurements are for 28 unique users, meaning that most of the observation are measurements taken at different time/day for the same user (participant)**

This table is not complete. 422 observations have missing values for functional_changes_index, circulatory_efficiency, or kerdo_vegetation_index, while 438 have missing values for the robinson_index. Deletion of these columns may be necessary since they are missing values for most of the observation.

#### Heart Rate
The heart rate table contains 523783 entries in total. It's important to note that while there are 523783 entries, there are multiple entries for the same user (i.e. one user had their heart rate read at different times which led to the multiple recordings). In actuality, if we combine the duplicate participant entries we see that there are 79 distinct user codes representing 79 participants.

The numerical features in the column are heart_rate and is_resting. heart_rate represents a heart beat recorded using a method they refer to as "hotoplethysmography (PPG)" in the text of the README in the original repository. The other numerical feature, is_resting is a value of either 0 or 1 representing if the user was at rest at the time of the measured recording.

The categorical features are the user_code, the unique ID referring to a patient, and datetime which consists of the exact date and time the heartbeat was recorded

#### HRV Measurements
The HRV table contains 3245 observations of users' (participants) HRV measurements. To be specific, these observation are for **185 unique users**

The numerical features represent the HRV measurements (heart rate during measurement, average time between each heartbeat in ms (meanrr), difference between highest and lowest cardio interval values in s (mxdmn), std of normal hearbeat intervals in ms (sdnn) etc...).

The categorical features represent the user id, the unique measurement id (rr_code: there are 3245 unique measurement ids corresponding to the total number of observations), the date and time of the measurement, tags (csv of single words, assigned by the user to describe their state during the measurement. Seems optional. Some users included a lot of words, while others didn't include any descriptive words), rr_data (intervals in ms b/w consecutive hearbeats as a csv). **Since we have the average rr, dropping the rr_data column may be a good option to consider**

The table seems complete, except for the features how_sleep (1779 missing values) and tags (1044 missing values). We may consider dropping the how_sleep column since +50% of observations don't have a value associated with it. Before dropping the tags column, we should consider compiling a list of tags for each of the 185 unique users as we may be able to fill in the gaps and get a better general overview of the user's mental/physical state as they reported it. 

#### Participants
The participants table contains 185 unique entries each representing a different particpant. For this dataset, the user_code, gender and age_range columns are all completely filled out, but the city, country, height, weight, and symptoms_onset columns contain missing entries.

The numerical columns consist of height and weight, which refer to a height measured in centimeters and weight measured in kilograms.

The categorical features are the user_code of the participant, their gender (referring only to 'm' or 'f'), city, country, and the onset of their symptoms.

#### Scales Description
The Scales Decription table contains 148 observations of different scales used to describe different states of the users' mental and physical being. Each scale is associated with a description, value denoting the intensity of the state the user is experiencing, and meaning to decipher the value associated with that scale. For example, the scale S_COVID_COUGH indicates a user experiencing coughing as a result of covid, where the values range from 1 (no cough) to 6(indicating extremely sever coughing)

The numerical features represent the value that corresponds to each scale (e.g intensity of the symptom experienced by the participant).

The categorical features represent the scale's name, description, and its meaning (given the associated value).

The table does not have any missing data, which is understandable since it represents the key to understand what the scales and their values mean when occuring in other tables. *We may only use this table to integrate the scale description in the Surveys table.*

#### Sleep

The sleep table consists of 425 entries, which are not unique and in actuality only represent 10 participants. This dataset also has mostly missing data, apart from the user_code, day, sleep_begin, sleep_end, and sleep_duration columns which are complete.

The categorical data are the user_code, day, sleep_begin (a date and time), and sleep_end (also a date and time).

The numerical data are sleep_duration which is the difference between sleep_end and sleep_begin in seconds. sleep_awake_duration which is a time in seconds of if a user woke up in the middle of their sleep. sleep_rem_duration representing the time they were in REM sleep. sleep_light_duration when they were not in REM or deep sleep. sleep_deep_duration when they weren't in light or REM sleep. A pulse_min, pulse_max and pulse_average representing their lowest, highest and average pulses. 

#### Surveys
The surveys table contains 2259 observations of users' (participants) reporting their physical/mental state (e.g whether or not they have diabetes). This table represents the reporting of **111 unique participants**. The scale, value, and text features correspond to the Scale, Value, and Meaning in the Scales_description table. 

The numerical and categorical features are similar to the scales description table. The only difference is the categorical feature created_at, which indicated the date on which the reporting happened. **It is interesting to note that the date reporting here is different than other tables, in the sense that it only indicates the date and not time of the observation**

This table does not contain any missing values, and no imputation or deletion of rows is necessary. **However, deleting the value or text row might be necessary since they convey similar information given the scale (although the text column is more descriptive).**

#### Wearables
The wearables table consists of 3098 non-unique entries representing 79 participants. Much of this dataset contains missing data.

The numerical features of this table are the user_code and the day columns.

The categorical features are resting_pulse, pulse_average, pulse_min, pulse_max which relate to the different factors of the user's pulse rate. An average_spo2_value column which is the average oxygen saturation in the blood. an average body temperature, total standing hours, total steps count, distance traveled, step speed, number of flights of stairs climbed, active calories burned, basal calories burned, total calories burned, an average of exposure time to headphones, and average time of environment exposure

#### Weather
The Weather table contains 1717 observations of weather conditions for users' (participants) locations aggregated by day. These observations are for **104 unique users**. 

The numerical features represent weather conditions such as the avg temperature in celsius for that day, atmospheric pressure, precipitation intensity (in pct), and clouds (coverage in pct).

The categorical features represent the user id and the day on which the observation was made. Only the date (and not time) was included since all these observations have been aggregated by day.

This table is complete, and no imputation or deletion of rows/columns is necessary.

### Descriptive Statistics

In [138]:
#Exclude the Scales Description table as discussed above 
try:
    del df_list[df_list_names.index('df_scales_description')]
    df_list_names.remove('df_scales_description')
except ValueError:
    print("Element already deleted from list")

In [ ]:
#Descriptive Statistics for numeric features
for i, df_name in enumerate(df_list_names):
    print(f'\nDescriptive Statistics for {df_name}')
    print('-'*15)
    numeric_columns = df_list[i].select_dtypes(include=[np.number]).columns
    print('Central Tendency Measures:')
    print(df_list[i][numeric_columns].describe().loc[['mean', '50%']])
    print('\nDispersion Measures:')
    print(df_list[i][numeric_columns].describe().loc[['std', 'min', 'max']])

    #Check for distribution normality (skewness and kurtosis)
    print('\nDistribution Measures:')
    print('-'*15)
    print(df_list[i][numeric_columns].skew())
    print(df_list[i][numeric_columns].kurtosis())
    print('\n')
    

### Interpretation

#### Blood Pressure 
- Most of our numerical values have means and medians relatively close to each other, indicating a roughly symmetric distribution (e.g diastolic mean is 81.228849 and median is 82)

- It is interesting to note the value range of the circulatory efficiency here, where the min 1300 and max value is 7875. *It might be interesting to investigate the existence of a relationship between the fluctuation of the circulatory efficiency with covid symptoms

- Our means and medians for all features are close in value, indicating a roughly summetric distribution. However, it is interesting to note the skewness values for diastolic, for instance, indicating a left skewed distribution. SImilarly, circulatory_efficiency is right skewed (positive skewness value indicating right skew). None of the features have a normal distribution of values given their kurtosis value.

#### HRV Measurements
- Close to half of the numerical features have means and medians relatively close to each other, indicating a roughly symmetric distribution. However, by checking the skewness value, we can see that almost all of the features have a right skewed distribution. Only 3 features, namely how_feel, how_mood, and how_sleep have a symmetrical distribution. Most of the features do not have a normal distribution given the kurtosis value

- It is interesting to note tha the mean of how_feel and how_mood features (answers to "How do you feel physically?" in the post-measurement survey, and answers to "How is your mood?" in the post-measurement survey respectively) is negative, indicating that on average most of the 185 participants had a bad mood and were not feeling great physically

#### Surveys

- Most features in this table are categorical. However, it is interesting to note how the values of the mean (2.364763) and median (2.0). Since the scales are sometimes boolean, the mean and median of the value feature do not provide interesting insights. *It would be interesting to see the mean and median for the 'negative' scales (i.e non boolean scales indicating the presence of symptom )*

#### Weather
- The mean and values of almost all features are close in value (except for precipitation intensity) indicating a roughly symmetrical distribution. It is interesting to note the min and max values (-14C and 44C respectively) for the average temperature feature. Since the std is around 7 degrees celsius, this might indicate some extreme values in the data that might cause skewness. 

=> We might need to normalize most of the numerical features in order to ensure that different scales contribute equally to any model that might be built for this dataset.

### Data Quality

- Checking for duplicated rows or inconsistent values
- Checking for outliers or extreme values that need attention

In [ ]:
#Data Quality
for i, df_name in enumerate(df_list_names):
    print(f'\nData Quality for {df_name}')
    print('-'*15)
    print(f'Duplicated Rows: {df_list[i].duplicated().sum()}')
    print('Checking for Inconsistent Values:')
    print(df_list[i].apply(lambda x: x.value_counts().index[0]).to_frame('most_frequent_values'))

- It is interesting to note how across all tables, the most frequent measurement days are really close, given us insights on when most of the data was collected (May 2020)
- There are no duplicated rows across all of the tables
- It is interesting to note how most survey obsertvations had a user not experiencing the symptom, indicating that most participants did not experience covid symptoms
- By checking the most frequent value for every feature across our tables, we were able to confirm that there are no outliers or extreme values that need our attention

### Variable Relationships

In [ ]:
#Variable Relationships 
#Exclude the surveys table as well since it only has one numeric value
for i, df_name in enumerate(df_list_names):
    if (df_name!='df_surveys'):
        numeric_columns = df_list[i].select_dtypes(include=[np.number]).columns
        print(f'\nVariable Relationships in {df_name}')
        print('-'*15)
        print('Correlation Matrix')
        correlation_matrix = df_list[i][numeric_columns].corr()

        #Visualize Heatmap
        plt.figure(figsize=(10, 8))
        sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f', linewidths=0.5)
        plt.title(f'Correlation Matrix Heatmap for {df_name}')
        plt.show()

### Interpretation

#### Blood Pressure
- It is interesting to see how the functional_changes_index (assessment of how well the body can adapt to stressors)and the robinson_index (the capacity of the cardiovascular system based on systolic blood pressure) are positively correlated variables, indicating that the capacity of the cardiovascular system increases as the body adapts better to stressors (vice-versa)
- It is also interesting to see how the kerdo_vegetation_index (balance between stress and recovery based on blood pressure) and the diastolic blood pressure are negatively correlated, indicating the one's balance between stress and recovery worsens as the blood pressure increases.

#### HRV Measurements
- The positive correlation between mxdmn (difference b/w highest and lowest cardio interval values), dnn, rmssd (root mean square of successive differences for consecutive intervals), and pnn50 (percent of RR-intervals that fall outside a 50 ms range of the average) since they all depend on the value of the cardio interval values (hearbeat interval). *This correlation indicates that creating a derived feature or interaction term combining all these features is a decision to explore*
- The negative correlation between bpm and meanrr is also expected since one is a measurements of beats per minute and the other is a measurement of the mean time between each heartbeat. *We should drop one of the features during our feature selection*
- The meanrr and the mode also represent similar measurements. *We should drop one of the features during the selection*
It is interesting to see how the how_feel and how_mood are positively correlated, which makes sense to some extent. Not feeling great physically will affect the person's mood. *It might interesting to explore combining both of these features* 

### Further Exploration of different tables

### Heart Rate CSV

In [ ]:
# read the csv and show how many rows are in the df
print("Number of entries for heart_rate.csv: " + str(df_heart_rate.shape[0]))
print('-' * 100)

# combine the rows based on duplicate user ids to show how many participants were part of this dataset
df_combined = df_heart_rate.drop(columns='datetime').groupby('user_code').mean().reset_index()
print("Number of particpants in this dataset: " + str(df_combined.shape[0]))
print('-' * 100)

# add a day column to the dataframe which shows what day of the week the heart rate was scanned
df_heart_rate['day'] = pd.to_datetime(df_heart_rate['datetime']).dt.day_name()

#display a few rows of the dataset to show the new column
print("Dataframe with new 'day' column:")
print(df_heart_rate.head())
print('-' * 100)

most_common_day = df_heart_rate['day'].value_counts()
print("Number of entries recorded by day of the week: ")
print(most_common_day)
print('-' * 100)

# print the average heartrate
print("Average heart rate amongst all entries: {:0.2f}" .format(df_heart_rate["heart_rate"].mean()))
print('-' * 100)
# get the rows for resting vs not resting
is_resting = df_heart_rate.query("is_resting == 1")
is_not_resting = df_heart_rate.query("is_resting == 0")

# show the number of participants resting vs not resting
print("Number of entries at rest vs not at rest: " + str(is_resting.shape[0]) + " vs " + str(is_not_resting.shape[0]))
print('-' * 100)
# show the average heart rate for participants at rest vs not at rest
print("Average heart rate at rest: {:0.2f}" .format(is_resting["heart_rate"].mean()))
print("Average heart rate not at rest: {:0.2f}" .format(is_not_resting["heart_rate"].mean()))

#### Interpretation

As we should expect, the resting heart rate is lower than the heart rate not at rest, and the average heart rate amongst the 79 participants was in the normal range (generally expected to be between 60-100 bpm). 
We created a new feature called days, which records the day of the week during which the data was recorded to have a better understanding of the data collection process. We can see the most common days heartrate was recorded were Tuesday and Sunday and participants recorded their data less frequently on Fridays and Saturdays.

### Participants CSV

In [ ]:
ave_age_map = {"18-24": 21, "25-34": 29.5, "35-44": 39.5, "45-54": 49.5, "55-64": 59.5, "65-74": 69.5}
ave_ages = df_participants["age_range"].map(ave_age_map)

#get the counts for each gender tested
genders = df_participants["gender"].value_counts(ascending=True)

#create a bar plot
genders.plot(kind="bar")
plt.xlabel('Gender')
plt.ylabel('# Participants')
plt.title('Number of Participants by Gender')

#get the current axis
ax = plt.gca()

#make the x-axis labels horizontal for easier readability
ax.set_xticklabels(genders.index, rotation=0)
plt.show()


#get the counts for each gender tested
ave_age_counts = df_participants["age_range"].value_counts()
ave_age_counts = ave_age_counts.sort_index(key=lambda x: x.str.split('-').str[0].astype(int))

#create a bar plot
ave_age_counts.plot(kind="bar")
plt.xlabel('Age')
plt.ylabel('# Participants')
plt.title('Number of Participants by Age')

#get the current axis
ax = plt.gca()

#make the x-axis labels horizontal for easier readability and apply a textwrap so labels don't overlap
ax.set_xticklabels(ave_age_counts.index, rotation=0)
plt.show()

males =  df_participants.query("gender == 'm'")
females =  df_participants.query("gender == 'f'")

print('-' * 100)
heights = males["height"].dropna()
ave_height = heights.mean()
print("Average height for male participants: " + str(ave_height))

weights = males["weight"].dropna()
ave_weight = weights.mean()
print("Average weight for male participants: " + str(ave_weight))

print('-' * 100)
heights = females["height"].dropna()
ave_height = heights.mean()
print("Average height for female participants: " + str(ave_height))

weights = females["weight"].dropna()
ave_weight = weights.mean()
print("Average weight for female participants: " + str(ave_weight))
print('-' * 100)

#Show how many unique cities and countries participants come from
cities = df_participants['city'].replace('', pd.NA)
cities.dropna(inplace=True)
print("How many unique cities participants came from: " + str(cities.unique().shape[0]))

countries = df_participants['country'].replace('', pd.NA)
countries.dropna(inplace=True)
print("How many unique countries participants came from: " + str(countries.unique().shape[0]))

onset_no_na = df_participants['symptoms_onset'].replace('', pd.NA)
percent_onset = onset_no_na.dropna().shape[0] / df_participants.shape[0]

print('-' * 100)
print("Percentage of participants that reported the date they started showing symptoms: {:0.2f}" .format(percent_onset))

#### Interpretation

For the participants.csv file, we decided to first look at the average age across all participants, which was about 40. We looked at the number of male vs female participants, of which we found there were about about 1.76 times as many female participants in the dataset. We can see that people ages 35-44 were the most common and most particpants were between 25 and 54. Of the male participants we found an average height of close to 179 cm (about 5'11") and an average weight of about 81 kg (178 lbs) and for female participants we found an average height of 165 cm (approx. 5'5") and average weight of about 76 kg (167 lbs). Participants were gathered from 116 different cities across 25 separate countries. However, for both the city and country columns, there were missing entries so there could be more cities and countries which were not listed. About 79% of participants reported a date in which they started symptoms. In terms of data quality, this column is somewhat ambiguous because without more information we can't determine with certainty whether a missing row for the onset of symptoms refers to a person that didn't have COVID, or if they just didn't report when their symptoms began.

### Sleep CSV
Notes: sleep_duration was found to be the difference between sleep_begin and sleep_end, which is why it was not included

In [ ]:
# remove the categorical columns for dimensionality reduction as we will not be using the categorical data for our EDA
sleep_df = df_sleep.drop(columns=['day', 'sleep_begin', 'sleep_end'])

# show how many entries are in the df
num_entries = sleep_df.shape[0]
print("Number of entries in total: " + str(num_entries))
print('-' * 100)

# show how many participants these entries correspond to
df_combined = sleep_df.groupby('user_code').mean().reset_index()
print("Number of particpants in this dataset: " + str(df_combined.shape[0]))
print('-' * 100)

# show the percentage of the df that each column represents and the details for the user
print("Percentage of the dataset each column represents: ")
for i in range(1, len(sleep_df.columns)):
    col_name = sleep_df.columns[i]
    col = sleep_df.dropna(subset=[col_name])
    print(" - " + col_name + " {:0.0f}" .format((col.shape[0]/num_entries) * 100) + "%")
    
    if i < 2:
        continue

    col_combined = col.groupby('user_code').mean().reset_index()
    for j in range(col_combined.shape[0]):
        if i < 6:
            print("     * Average " + col_name + " for user " + col_combined.iloc[j,0] + ": {:0.2f} hrs" .format(col_combined.iloc[j,i] / 3600.0))
        else:
            print("     * Average " + col_name + " for user " + col_combined.iloc[j,0] + ": {:0.2f}" .format(col_combined.iloc[j,i]))

print('-' * 100)

# get the sleep duration column, drop NaNs and convert the time to hours
sleep_duration = sleep_df["sleep_duration"].dropna() / 3600.0

# create a box plot
sleep_duration.plot(kind="box", vert=False)

# modify the y-ticks for easier readability
current_y_ticks = plt.gca().get_yticklabels()
new_y_ticks = [tick.get_text().replace('_', '\n') for tick in current_y_ticks]
plt.gca().set_yticklabels(new_y_ticks)

plt.title('Sleep Duration Box Plot with Outliers')
plt.show()

#### Interpretation

We can see that of the 425 entries recorded in sleep.csv, the data corresponds to 10 participants. In terms of data completeness and data consistency, this csv has some downfalls. For all numerical columns apart from sleep_duration, the data recorded for each column was less than 6% of the entire dataframe. Which means at most, 6% of that column was recorded. Furthermore, for each column, the user for which the data was recorded was very inconsistent. For each individual column, there was at most 3 users per column represented by the recorded data (not including sleep_duration). This makes it very unreliable to truly assess the data in those columns.

For sleep_duration, we can see that 100% of that column was filled. From the box plot and the descriptive statistics printed out below the box plot  we can see that average sleep duration was just over 7 hours with a minimum of just over 15 minutes and a maximum of over 13 hours. 

### Wearables CSV

In [ ]:
# drop day column as we will not be using it
wearables_df = df_wearables.drop(columns=['day'])

# get the number of entries for the wearables df
num_entries = wearables_df.shape[0]
print("Number of entries in total: " + str(num_entries))
print('-' * 100)

# get the number of participants in this dataset
df_combined = wearables_df.groupby('user_code').mean().reset_index()
print('Number of particpants in this dataset: ' + str(df_combined.shape[0]))
print('-' * 100)

# only use data that has data which represents more than 50% of the dataset
significant_cols = []
for col_name in wearables_df.columns[1:]:
    col = wearables_df[col_name].dropna()
    if col.shape[0]/num_entries > 0.5:
        significant_cols.append(col_name)

# get the averages for the data from the remaining columns
for col_name in significant_cols:
    col = wearables_df[col_name].dropna()
    print("Average " + col_name + ": {:0.2f}" .format(col.mean()))

#### Interpretation

We can see that of the over 3000 entries, this dataset represents 79 participants. In terms of data completeness, of the 16 numerical columns contained in the dataset, only 7 of the columns consisted of data which represented over half of the entire dataset (the threshold we arbitrarily chose to consider a column complete enough to warrant studying). Interestingly, although both the heart rate and wearables tables recorded pulse, we can see that the average pulse rate reported by wearables is about 13 bpm lower than what was reported in the heart rate table. This could warrant further investigation in the different methods for measuring heart rate. Perhaps there are some inaccuracies in one of the methods. While steps count and basal and total calories are self-evident in what they are measuring, without more detail, distance is left ambiguous. In terms of data quality, curation of the dataset should've been more clear in marking the distance measurements. 

### More Data Visualization

In [146]:
df_list = [df_blood_pressure, df_heart_rate, df_hrv_measurements, df_participants, df_sleep, df_surveys, df_wearables, df_weather]
df_list_names = ["df_blood_pressure", "df_heart_rate", "df_hrv_measurements", "df_participants", "df_sleep", "df_surveys", "df_wearables", "df_weather"]

In [147]:
def scale_data(df, scaler):
    return scaler.fit_transform(df)

In [148]:
#Visualize distributions for numerical features for each table
def visualize_dist_for_num_features(df, df_name):
    df.hist(bins=25, figsize=(10,10), grid=False)
    plt.suptitle(f'Histograms of Numeric Features of {df_name}', y=0.93)
    plt.show()

#Box plots to identify outliers
def visualize_outliers(df, df_name, numeric_columns):
    #Box plots to identify outliers
   
    #scaling the data for better plotting
    scaler = StandardScaler()
    df_scaled = scaler.fit_transform(df[numeric_columns])
    plt.figure(figsize=(15,10))
    sns.boxplot(data=df_scaled)
    plt.title(f'Box Plots of Numeric Features of {df_name}')
    plt.xlabel('Feature')
    plt.ylabel('Value')
    plt.xticks(range(0, len(numeric_columns)), numeric_columns.to_list(), rotation=45, ha='right')
    plt.tight_layout()
    plt.show()

#### Blood Pressure

In [ ]:
numeric_columns = df_blood_pressure.select_dtypes(include=[np.number]).columns

visualize_dist_for_num_features(df_blood_pressure[numeric_columns], 'Blood Pressure')
visualize_outliers(df_blood_pressure, 'Blood Pressure', numeric_columns)


- There are few individual histograms in circulatory_efficiency, indicating two  extremely small distinct groups within the data, likely corresponding to some outliers or extreme values that were reported only a few times, as maybe a hardware malfunction. This same observation applies to all the other features. We can confirm this observation by looking at the box plot which plots the outliers. *We need to either remove these outliers and try to understand why they are occuring*.
- Some of these outliers clearly represent extreme values that can be removed from the data. As for the other outliers, depending on their count, we may just replace that value with the average value for that feature
- Some of our features like, circulatory_efficiency, are somewhat symmetric. However, most of our features distributions are skewed, requiring transformation, to normalize it.

#### HRV Measurements

In [ ]:
numeric_columns = df_hrv_measurements.select_dtypes(include=[np.number]).columns
visualize_dist_for_num_features(df_hrv_measurements[numeric_columns], 'HRV Measurements')
visualize_outliers(df_hrv_measurements, 'HRV Measurements', numeric_columns)

- Data has a quite a few outliers that can be categorized as extreme values. 
- Some features have symmetrical distributions, but non have a normal distribution. *We will need to normalize the data*

### Data Accuracy

We can safely assume that the data is accurate given the different steps taken by the researchers to ensure accuracy. We cannot cross check it against other sources since this data is proprietary. 

### Data Consistency

The data is mostly consistent across all sources. It is important to note how some tables keep track of the date followed by time, while others only include the date measurement.

It is also important to note how some people is the participants table have entered their birthdate in the symptom_onset column (The onset date of the symptoms of the disease. The format is MM/DD/YYYY.). *We will need to manually remove these participants from all tables*

### Data Integrity

In terms of data integrity, the researchers included data that is consistent in terms of having unique UUIDs, valid value ranges etc.

# Data Preprocessing

### Data Cleaning

Since our datasets do not have any duplicated observations, removing duplicates is not issue in our case.

To handle missing values, we can either use the averages of the values that are currently used for the features missing values or fill the missing values with the most frequent value (although this approach my introduce some bias in our datase, unless the most frequent value is close in value to the mean). Another approach is to drop the columns that mostly has missing values, such as the functional_change_index column in the Blood Pressure table (discussed in the Data Profiling section). 

In [ ]:
df_blood_pressure.head()

In [152]:
df_blood_pressure = df_blood_pressure.drop(columns=['functional_changes_index', 'circulatory_efficiency', 'kerdo_vegetation_index', 'robinson_index'])

In [ ]:
df_blood_pressure.head()

We will use the Wearables table to illustrate our strategy in handling outliers. We detect outliers via the Standard Deviation method, setting k = 3.  

In [ ]:
# get the z-scores for the sleep_duration column
sleep_df['Z-Score'] = (sleep_df["sleep_duration"] - sleep_df["sleep_duration"].mean()) / sleep_df["sleep_duration"].std()

# Find outliers with a z-score > 2 or < -2
sleep_df['Outlier'] = np.where((sleep_df['Z-Score'] > 3) | (sleep_df['Z-Score'] < -3), True, False)

# get the sleep duration column without outliersg, drop NaNs and convert the time to hours
df_without_outliers = sleep_df[sleep_df['Outlier'] == False]
sleep_duration = df_without_outliers["sleep_duration"].dropna() / 3600.0

# create a box plot
sleep_duration.plot(kind="box", vert=False)
plt.title('Sleep Duration Box Plot without Outliers')

# modify the y-ticks for easier readability
current_y_ticks = plt.gca().get_yticklabels()
new_y_ticks = [tick.get_text().replace('_', '\n') for tick in current_y_ticks]
plt.gca().set_yticklabels(new_y_ticks)

plt.show()

# show the number of outliers
print('-' * 100)
print('Number of Outliers: ' + str(sleep_df[sleep_df['Outlier'] == True].shape[0]))

# show statistics for sleep duration
print('-' * 100)
sleep_duration.describe()

### Data Transformation

In the participants table, we created a new feature called BMI (Body-mass index). We felt that keeping track of height and weight may not be as meaningful as keeping track of the BMI. After creating this feature, we dropped the columns. The formula to calculate the BMI can be found here (https://www.nhs.uk/health-assessment-tools/calculate-your-body-mass-index/calculate-bmi-for-adults)

In [ ]:
#create a new feature, bmi, for the participants table
df_participants["bmi"] = df_participants["weight"] / ((df_participants["height"] / 100.0)**2)
df_participants.drop(columns=['weight', 'height'])
print(df_participants["bmi"].head())

We will perform dimensionality reduction on the HRV measurements table using PCA. If we were to use this data to answer a question, we would use it to answer whether or not the participant has covid-19 based on their hrv measurements. Since the target here would be binary, linearity is assumed. As we saw in the visualization plots for the HRV measurements table, most of the numerical features are positively skewed. To normalize the data, we will use log transformation

In [ ]:
df_hrv_measurements.head()

In [157]:
#Get the numerical features to normalize and exclude the how_feel how_mood and how_sleep 
numeric_columns = df_hrv_measurements.select_dtypes(include=[np.number]).columns
#We are exlusing bpm and meanrr to focus on the features that are highly correlated here
df_hrv_numeric = df_hrv_measurements[numeric_columns].drop(columns=['how_feel', 'how_mood', 'how_sleep', 'bpm', 'meanrr'])

# Apply log transformation
df_hrv_log = np.log1p(df_hrv_numeric)

Now that the assumptions for PCA are met, we can implement the method and reduce the table's dimensions. Used this article to implement the PCA: https://www.geeksforgeeks.org/implementing-pca-in-python-with-scikit-learn/#
Used this post to figure out the most optimal number of components: https://stackoverflow.com/questions/53802098/how-to-choose-the-number-of-components-pca-scikitliear

In [ ]:
from sklearn.decomposition import PCA

#Perform PCA on the transformed features (the numeric features)
pca = PCA()
df_pca = pca.fit_transform(df_hrv_log)
cumulative_variance_ratio = np.cumsum(pca.explained_variance_ratio_)

# Plot the cumulative explained variance ratio
plt.plot(range(1, len(cumulative_variance_ratio) + 1), cumulative_variance_ratio, 'bo-')
plt.xlabel('Number of Components')
plt.ylabel('Cumulative Explained Variance Ratio')
plt.title('Explained Variance vs. Number of Components')
plt.show()

As we can see, we only need about 4 principal components to explain about 95% of the variance. Additional components do no necessarily add much to the explainability. Thus, we will use 4 principal components

In [ ]:
# Apply PCA with the chosen number of components
pca = PCA(n_components=4)
df_pca_final = pca.fit_transform(df_hrv_log)

# Create a new dataframe with the PCA results
df_pca_final = pd.DataFrame(data=df_pca_final, columns=[f'PC{i+1}' for i in range(4)])

#Combine the reduced numeric features with the columns we excluded earlier such as rr_data and tags
#We are not including the meanrr here since bpm captures the same data pretty much to avoid correlation
columns_to_combine = ['rr_data', 'tags', 'how_sleep', 'how_feel', 'how_mood', 'bpm']
df_hrv_measurements_reduced = pd.concat([df_pca_final, df_hrv_measurements[columns_to_combine]], axis=1)

#Print the columns of the df_hrv_measurements
df_hrv_measurements_reduced.head()

### Interpretation of Dimensionality Reduction

We can plot the correlation matrix to see if we did better after reducing our features. We can also plot the histograms of these features to have better visualization.

In [ ]:
numeric_columns = df_hrv_measurements_reduced.select_dtypes(include=[np.number]).columns
#histograms
visualize_dist_for_num_features(df_hrv_measurements_reduced[numeric_columns], 'HRV Measurements')

#Correlation heatmap
print('Correlation Matrix')
correlation_matrix = df_hrv_measurements_reduced[numeric_columns].corr()

#Visualize Heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f', linewidths=0.5)
plt.title(f'Correlation Matrix Heatmap for {df_name}')
plt.show()

We can see that our reduced features dealt with the multicorrelation problem we had earlier. Given how the features reduced were correlated to the bpm, it is interesting to see how some of these new features are still correlated with the bpm.